# 1. Initialize

### Libraries and Spark context

In [16]:
import pyspark
from pyspark.sql import SparkSession
from probetrip.vehicle_performance import VehiclePerformance

In [17]:
spark = SparkSession.builder.master("yarn").appName("5_speed_acc") \
        .config("spark.executor.memory", "4g").config("spark.driver.memory", "4g") \
        .config("spark.dynamicAllocation.minExecutors", "1").config("spark.dynamicAllocation.maxExecutors", "3") \
        .config("spark.sql.catalogImplementation", "hive") \
        .config("metastore.catalog.default", "hive") \
        .enableHiveSupport() \
        .getOrCreate()

24/07/16 11:39:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/07/16 11:39:57 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [23]:
spark.stop()

### Required parameter

**Trip table structure** (input table)
<table style='float:left;'>
    <tr>
        <th style='text-align: left;'>Name</th>
        <th style='text-align: left;'>Data type</th>
        <th style='text-align: left;'>Example (Stay)</th>
        <th style='text-align: left;'>Example (Move)</th>
    </tr>
    <tr>
        <td style='text-align: left;'>id</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>+1indEOKr/ikPVrJQTVjw4FGxBE</td>
        <td style='text-align: left;'>+1indEOKr/ikPVrJQTVjw4FGxBE</td>
    </tr>
    <tr>
        <td style='text-align: left;'>trip_date</td>
        <td style='text-align: left;'>date </td>
        <td style='text-align: left;'>2023-01-01</td>
        <td style='text-align: left;'>2023-01-01</td>
    </tr>
    <tr>
        <td style='text-align: left;'>trip_sequence</td>
        <td style='text-align: left;'>integer</td>
        <td style='text-align: left;'>1</td>
        <td style='text-align: left;'>1</td>
    </tr>
    <tr>
        <td style='text-align: left;'>trip_sequence</td>
        <td style='text-align: left;'>integer</td>
        <td style='text-align: left;'>1</td>
        <td style='text-align: left;'>1</td>
    </tr>
    <tr>
        <td style='text-align: left;'>mobility_type</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>STAY</td>
        <td style='text-align: left;'>MOVE</td>
    </tr>
    <tr>
        <td style='text-align: left;'>passenger</td>
        <td style='text-align: left;'>integer</td>
        <td style='text-align: left;'>0</td>
        <td style='text-align: left;'>1</td>
    </tr>
    <tr>
        <td style='text-align: left;'>total_distance (km)</td>
        <td style='text-align: left;'>float</td>
        <td style='text-align: left;'>0.0</td>
        <td style='text-align: left;'>209.57</td>
    </tr>
    <tr>
        <td style='text-align: left;'>total_time (minutes)</td>
        <td style='text-align: left;'>float</td>
        <td style='text-align: left;'>318.53</td>
        <td style='text-align: left;'>390.3</td>
    </tr>
    <tr>
        <td style='text-align: left;'>overall_speed (km/h)</td>
        <td style='text-align: left;'>float</td>
        <td style='text-align: left;'>318.53</td>
        <td style='text-align: left;'>32.22</td>
    </tr>
    <tr>
        <td style='text-align: left;'>average_speed (km/h)</td>
        <td style='text-align: left;'>float</td>
        <td style='text-align: left;'>0.0</td>
        <td style='text-align: left;'>28.8</td>
    </tr>
    <tr>
        <td style='text-align: left;'>start_time</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>00:02:49</td>
        <td style='text-align: left;'>05:21:21</td>
    </tr>
    <tr>
        <td style='text-align: left;'>end_time</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>05:21:21</td>
        <td style='text-align: left;'>11:51:39</td>
    </tr>
    <tr>
        <td style='text-align: left;'>total_points</td>
        <td style='text-align: left;'>integer</td>
        <td style='text-align: left;'>121</td>
        <td style='text-align: left;'>283</td>
    </tr>
    <tr>
        <td style='text-align: left;'>sub_district</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>Lam Phak Kut</td>
        <td style='text-align: left;'></td>
    </tr>
    <tr>
        <td style='text-align: left;'>district</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>Thanyaburi</td>
        <td style='text-align: left;'></td>
    </tr>
    <tr>
        <td style='text-align: left;'>province</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>Pathum Thani</td>
        <td style='text-align: left;'></td>
    </tr>
    <tr>
        <td style='text-align: left;'>point_list</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>[[14.032970,100.786000]]</td>
        <td style='text-align: left;'>[["05:21:21",14.03297,100.786],["05:22:21",14.0346,100.78606], ..., ["11:51:39",14.03296,100.78601]]</td>
    </tr>
    <tr>
        <td style='text-align: left;'>partition_month</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>202301</td>
        <td style='text-align: left;'>202301</td>
    </tr>
</table>

**Speed Acceleration table structure** (output table)
<table style='float:left;'>
    <tr>
        <th style='text-align: left;'>Name</th>
        <th style='text-align: left;'>Data type</th>
        <th style='text-align: left;'>Example</th>
    </tr>
    <tr>
        <td style='text-align: left;'>id</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>+1indEOKr/ikPVrJQTVjw4FGxBE</td>
    </tr>
    <tr>
        <td style='text-align: left;'>trip_date</td>
        <td style='text-align: left;'>date</td>
        <td style='text-align: left;'>2023-01-01</td>
    </tr>
    <tr>
        <td style='text-align: left;'>hour</td>
        <td style='text-align: left;'>integer</td>
        <td style='text-align: left;'>5</td>
    </tr>
    <tr>
        <td style='text-align: left;'>lat</td>
        <td style='text-align: left;'>float</td>
        <td style='text-align: left;'>14.03589</td>
    </tr>
    <tr>
        <td style='text-align: left;'>lon</td>
        <td style='text-align: left;'>float</td>
        <td style='text-align: left;'>100.78168</td>
    </tr>
    <tr>
        <td style='text-align: left;'>speed (km/h)</td>
        <td style='text-align: left;'>float</td>
        <td style='text-align: left;'>14.7</td>
    </tr>
    <tr>
        <td style='text-align: left;'>acc (m/s<sup>2</sup>)</td>
        <td style='text-align: left;'>float</td>
        <td style='text-align: left;'>0.0090</td>
    </tr>
    <tr>
        <td style='text-align: left;'>partition_month</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>202301</td>
    </tr>
</table>

In [8]:
database_name = "probe_taxi"
trip_table = "trip_point"
speed_acc_table = "speed_acc"
start_date = '20230101'
end_date = '20230101'

# 2. Execution

In [10]:
vp = VehiclePerformance(spark, database_name, trip_table, speed_acc_table)

In [13]:
vp.generate_speed_acc(start_date, end_date)

24/07/12 10:48:36 WARN SimpleFunctionRegistry: The function cal_point_speed_acc replaced a previously registered function.


Generate Speed-Acceleration on 20230116 successfuly! 	 | Process time: 0.77 mins


Generate Speed-Acceleration on 20230117 successfuly! 	 | Process time: 0.25 mins


Generate Speed-Acceleration on 20230118 successfuly! 	 | Process time: 0.03 mins


Generate Speed-Acceleration on 20230119 successfuly! 	 | Process time: 0.13 mins


...

Generate Speed-Acceleration on 20230127 successfuly! 	 | Process time: 0.83 mins


Generate Speed-Acceleration on 20230128 successfuly! 	 | Process time: 0.81 mins


Generate Speed-Acceleration on 20230129 successfuly! 	 | Process time: 0.40 mins


Generate Speed-Acceleration on 20230130 successfuly! 	 | Process time: 0.79 mins


Generate Speed-Acceleration on 20230131 successfuly! 	 | Process time: 0.80 mins
